In [ ]:
!pip install numpy pandas tensorflow tensorflow-hub scikit-learn lightgbm xgboost catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import  Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.svm import SVR

In [ ]:

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

In [ ]:

data = pd.read_csv('/content/drive/MyDrive/corrosiondatasetformagnesiumalloys.csv')

In [ ]:

text_data = data['processing'].astype(str)
text_data1 = data['processing1'].astype(str)
text_data2 = data['second_phase'].astype(str)


text_features = embed(text_data)
text_features1 = embed(text_data1)
text_features2 = embed(text_data2)


text_features = np.round(text_features, 6)
text_features1 = np.round(text_features1, 6)
text_features2 = np.round(text_features2, 6)

In [ ]:

def create_autoencoder(input_dim, encoding_dim):
    input_layer = tf.keras.layers.Input(shape=(input_dim,))
    encoder_layer = tf.keras.layers.Dense(encoding_dim, activation='relu')(input_layer)
    decoder_layer = tf.keras.layers.Dense(input_dim, activation='sigmoid')(encoder_layer)
    autoencoder = tf.keras.Model(input_layer, decoder_layer)
    encoder = tf.keras.Model(input_layer, encoder_layer)
    return autoencoder, encoder

In [ ]:

def load_encoder(input_dim, encoding_dim, weight_file):
    autoencoder, encoder = create_autoencoder(input_dim, encoding_dim)
    encoder.load_weights(weight_file)
    return encoder


encoder = load_encoder(input_dim=text_features.shape[1], encoding_dim=5, weight_file='/content/drive/MyDrive/encoder_weights_2.h5')
encoder1 = load_encoder(input_dim=text_features1.shape[1], encoding_dim=5, weight_file='/content/drive/MyDrive/encoder1_weights_2.h5')
encoder2 = load_encoder(input_dim=text_features2.shape[1], encoding_dim=5, weight_file='/content/drive/MyDrive/encoder2_weights_2.h5')

In [ ]:

encoded_data = encoder.predict(text_features)
encoded_data1 = encoder1.predict(text_features1)
encoded_data2 = encoder2.predict(text_features2)


encoded_data = np.round(encoded_data, 6)
encoded_data1 = np.round(encoded_data1, 6)
encoded_data2 = np.round(encoded_data2, 6)

44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [ ]:


autoencoder, encoder = create_autoencoder(input_dim=text_features.shape[1], encoding_dim=5)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(text_features, text_features, epochs=10, batch_size=1)
encoder.save_weights('encoder.weights.h5')

autoencoder1, encoder1 = create_autoencoder(input_dim=text_features1.shape[1], encoding_dim=5)
autoencoder1.compile(optimizer='adam', loss='mse')
autoencoder1.fit(text_features1, text_features1, epochs=10, batch_size=1)
encoder1.save_weights('encoder1.weights.h5')

autoencoder2, encoder2 = create_autoencoder(input_dim=text_features2.shape[1], encoding_dim=5)
autoencoder2.compile(optimizer='adam', loss='mse')
autoencoder2.fit(text_features2, text_features2, epochs=10, batch_size=1)
encoder2.save_weights('encoder2.weights.h5')

encoded_data = encoder.predict(text_features)
encoded_data1 = encoder1.predict(text_features1)
encoded_data2 = encoder2.predict(text_features2)


encoded_data = np.round(encoded_data, 6)
encoded_data1 = np.round(encoded_data1, 6)
encoded_data2 = np.round(encoded_data2, 6)

Epoch 1/10
1402/1402 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0654
Epoch 2/10
1402/1402 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.0016
Epoch 3/10
1402/1402 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0014
Epoch 4/10
1402/1402 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0014
Epoch 5/10
1402/1402 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0013
Epoch 6/10
1402/1402 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0013
Epoch 7/10
1402/1402 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0012
Epoch 8/10
1402/1402 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0012
Epoch 9/10
1402/1402 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.0012
Epoch 10/10
1402/1402 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.0012
Epoch 1/10
1402/1402 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.0646
Epoch 2/10
1402/1402 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0015
Epoch 3/10
1402/1402 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.0013
Epoch 4/10
1402/1402 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.0013
Epoch 5/10
1402/1402 ━━━━━━━━━━━━

In [ ]:

other_features = data.drop(columns=['processing', 'processing1', 'pw', 'second_phase', 'Literature No', 'Compostion presentation' , 'DFT_diff' ,]).values
combined_features = np.concatenate((encoded_data, encoded_data1, encoded_data2, other_features), axis=1)


Y = data['pw'].values

In [ ]:

X_train, X_test, Y_train, Y_test, train_indices, test_indices = train_test_split(combined_features, Y, data.index.values, test_size=0.3, random_state=42)


In [ ]:
from sklearn.svm import SVR

def custom_asymmetric_train(y_true, y_pred):
    residual = (y_true - y_pred).astype("float")
    grad = np.where(residual > 0, -2 * residual, -1 * residual)
    hess = np.where(residual > 0, 2, 1)
    return grad, hess

def custom_asymmetric_objective(y_true, y_pred):
    grad, hess = custom_asymmetric_train(y_true, y_pred)
    return grad, hess


lgb_model = lgb.LGBMRegressor(objective=custom_asymmetric_objective, metric='rmse', n_estimators=600, learning_rate=0.1)


lgb_model.fit(X_train, Y_train, eval_metric=lambda y_true, y_pred: custom_asymmetric_valid(y_true, y_pred))


initial_pred = lgb_model.predict(X_train)


svr_model = SVR(kernel='rbf', C=900, gamma=0.3, epsilon=0.01, tol=0.3)


initial_pred_svr = svr_model.fit(X_train, Y_train).predict(X_train)


residuals = Y_train - initial_pred_svr
weights = np.where(residuals > 0, 2, 1)


X_train_weighted = np.repeat(X_train, weights.astype(int), axis=0)
Y_train_weighted = np.repeat(Y_train, weights.astype(int), axis=0)


svr_model.fit(X_train_weighted, Y_train_weighted)




kernel1 = C(1.0, (1e-3, 1e3)) * RBF(1, (1e-2, 1e2))
gpr_model = GaussianProcessRegressor(kernel=kernel1, n_restarts_optimizer=10, alpha=0.01)


gpr_model.fit(X_train, Y_train)
initial_predictions = gpr_model.predict(X_train)
initial_residuals = Y_train - initial_predictions


weights1 = np.where(initial_residuals > 0, 2, 1)


X_train_weighted1 = np.repeat(X_train, weights1.astype(int), axis=0)
Y_train_weighted1 = np.repeat(Y_train, weights1.astype(int), axis=0)

gpr_model.fit(X_train_weighted1, Y_train_weighted1)



lgb_pred_train = lgb_model.predict(X_train)
svr_pred_train = svr_model.predict(X_train)
gpr_pred_train = gpr_model.predict(X_train)


lgb_pred_test = lgb_model.predict(X_test)
svr_pred_test = svr_model.predict(X_test)
gpr_pred_test = gpr_model.predict(X_test)




final_pred = (lgb_pred_test + svr_pred_test + gpr_pred_test)/3


mse = mean_squared_error(Y_test, final_pred)
print(f"Mean Squared Error: {mse}")


r2 = r2_score(Y_test, final_pred)
print(f"R-squared Score: {r2}")


comparison_df = pd.DataFrame({"True Values": Y_test, "Predicted Values": final_pred})
print(comparison_df.head(10))

[LightGBM] [Info] Using self-defined objective function
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000334 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2465
[LightGBM] [Info] Number of data points in the train set: 981, number of used features: 27
[LightGBM] [Info] Using self-defined objective function


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


Mean Squared Error: 0.20997131537386504
R-squared Score: 0.8302811134797843
   True Values  Predicted Values
0     0.397940          0.249753
1     3.289366          3.128559
2     1.304491          1.133424
3     0.071882          0.556489
4     0.975432          0.894164
5     1.574031          1.681217
6     0.041393         -0.182484
7     1.743823          1.891601
8     0.317854          0.376452
9     1.005627          1.053576


In [ ]:
import joblib


joblib.dump(lgb_model, '/content/drive/MyDrive/lgb_model_2.pkl')
joblib.dump(svr_model, '/content/drive/MyDrive/svr_model_2.pkl')
joblib.dump(gpr_model, '/content/drive/MyDrive/gpr_model_2.pkl')

['/content/drive/MyDrive/gpr_model_2.pkl']

In [ ]:
import joblib

In [ ]:

batch_size = 10000


def read_in_chunks(file_path, chunksize=batch_size):
    for chunk in pd.read_csv(file_path, chunksize=chunksize):
        yield chunk


result1 = []

for chunk in read_in_chunks('/content/drive/MyDrive/bottom20_400_24h_350_16on1.csv'):

    new_text_data = chunk['processing'].astype(str)
    new_text_data1 = chunk['processing1'].astype(str)
    new_text_data2 = chunk['second_phase'].astype(str)


    new_text_features = embed(new_text_data)
    new_text_features1 = embed(new_text_data1)
    new_text_features2 = embed(new_text_data2)


    new_text_features = np.round(new_text_features, 6)
    new_text_features1 = np.round(new_text_features1, 6)
    new_text_features2 = np.round(new_text_features2, 6)


    new_encoded_data = encoder.predict(new_text_features)
    new_encoded_data1 = encoder1.predict(new_text_features1)
    new_encoded_data2 = encoder2.predict(new_text_features2)

    new_encoded_data = np.round(new_encoded_data, 6)
    new_encoded_data1 = np.round(new_encoded_data1, 6)
    new_encoded_data2 = np.round(new_encoded_data2, 6)


    new_other_features = chunk.drop(columns=['processing', 'processing1', 'second_phase']).values

    new_other_features = np.round(new_other_features, 6)


    print("new_encoded_data shape:", new_encoded_data.shape)
    print("new_encoded_data1 shape:", new_encoded_data1.shape)
    print("new_encoded_data2 shape:", new_encoded_data2.shape)
    print("new_other_features shape:", new_other_features.shape)


    if new_encoded_data.shape[0] == new_encoded_data1.shape[0] == new_encoded_data2.shape[0] == new_other_features.shape[0]:

        new_combined_features = np.concatenate((new_encoded_data, new_encoded_data1, new_encoded_data2, new_other_features), axis=1)
        loaded_lgb_model = joblib.load('/content/drive/MyDrive/lgb_model_2.pkl')
        loaded_svr_model = joblib.load('/content/drive/MyDrive/svr_model_2.pkl')
        loaded_gpr_model = joblib.load('/content/drive/MyDrive/gpr_model_2.pkl')


        svr_predictions = loaded_svr_model.predict(new_combined_features)


        lgb_predictions = loaded_lgb_model.predict(new_combined_features)


        gpr_predictions = loaded_gpr_model.predict (new_combined_features)


        new_predictions = (svr_predictions + lgb_predictions + gpr_predictions)/3
        #new_predictions = (svr_predictions +  lgb_predictions)/2




        chunk['predicted_pw'] = new_predictions


        result1.append(chunk[['predicted_pw']])
    else:
        raise ValueError("所有输入数组的样本数量必须相同")

final_result1 = pd.concat(result1)

final_result1.to_csv('/content/drive/MyDrive/predicted_results_new_2024_08_21.csv', index=False)

print("预测完成，结果已保存至: predicted_results_new.csv")
print(final_result1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
new_encoded_data shape: (20, 5)
new_encoded_data1 shape: (20, 5)
new_encoded_data2 shape: (20, 5)
new_other_features shape: (20, 18)


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


预测完成，结果已保存至: predicted_results_new.csv
    predicted_pw
0       4.247379
1       4.307586
2       3.914362
3       4.289357
4       4.078225
5       3.503859
6       3.607327
7       3.776287
8       4.070184
9       3.685367
10      3.853826
11      3.545687
12      3.886349
13      3.613883
14      3.763848
15      3.548531
16      3.679235
17      3.937093
18      3.881204
19      3.758857


In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print(final_result1)

    predicted_pw
0       4.247379
1       4.307586
2       3.914362
3       4.289357
4       4.078225
5       3.503859
6       3.607327
7       3.776287
8       4.070184
9       3.685367
10      3.853826
11      3.545687
12      3.886349
13      3.613883
14      3.763848
15      3.548531
16      3.679235
17      3.937093
18      3.881204
19      3.758857


In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print(final_result1)

    predicted_pw
0       4.247379
1       4.307586
2       3.914362
3       4.289357
4       4.078225
5       3.503859
6       3.607327
7       3.776287
8       4.070184
9       3.685367
10      3.853826
11      3.545687
12      3.886349
13      3.613883
14      3.763848
15      3.548531
16      3.679235
17      3.937093
18      3.881204
19      3.758857


In [ ]:
import pandas as pd

file_path = '/content/drive/MyDrive/new_predicted_alloy_dataforhighcorrosion_withimpurity_newDFT_updated_delete_final.csv'
df = pd.read_csv(file_path)
print(df[:1])

    Sn   Ga   In   Al   La   Mn   Zn    Y   Zr   Ca   Gd   Nd      Fe      Ni  \
0  0.5  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0078  0.0042   

       Cu      Si   Sr                                         processing  \
0  0.0044  0.0131  0.0  The alloy was prepared by the conventional gra...   

                                       second_phase  \
0  Magnesium alloy containing unknown second phase.   

                                         processing1   DFT_ave  
0  Immersion in the solution containing 137 mM Na...  0.406929  


In [ ]:
import pandas as pd


file_path = '/content/drive/MyDrive/new_predicted_alloy_dataforhighcorrosion_withimpurity_newDFT_updated_delete_final.csv'
df = pd.read_csv(file_path)


new_processing1_value = "Immersion in 3.5 wt.% NaCl solution at 25 °C for 2 h."
df['processing1'] = new_processing1_value
df = df.drop(columns=['processin1'])
output_file_path = '/content/drive/MyDrive/new_predicted_alloy_dataforhighcorrosion_withimpurity_newDFT_updated_delete_final.csv'
df.to_csv(output_file_path, index=False, encoding='utf-8-sig')

print("Processing1列修改完成并已保存为新的CSV文件。")

Processing1列修改完成并已保存为新的CSV文件。


KeyboardInterrupt: 

In [ ]:
print(df[:1])

    Sn   Ga   In   Al   La   Mn   Zn    Y   Zr   Ca   Gd   Nd      Fe      Ni  \
0  0.5  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0078  0.0042   

       Cu      Si   Sr                                         processing  \
0  0.0044  0.0131  0.0  The alloy was prepared by the conventional gra...   

                                       second_phase  \
0  Magnesium alloy containing unknown second phase.   

                                         processing1   DFT_ave  
0  Immersion in 3.5 wt.% NaCl solution at 25 °C f...  0.406929  


In [ ]:

batch_size = 10000


def read_in_chunks(file_path, chunksize=batch_size):
    for chunk in pd.read_csv(file_path, chunksize=chunksize):
        yield chunk


result1 = []

for chunk in read_in_chunks('/content/drive/MyDrive/new_predicted_alloy_dataforhighcorrosion_withimpurity_newDFT_updated_delete_final.csv'):

    new_text_data = chunk['processing'].astype(str)
    new_text_data1 = chunk['processing1'].astype(str)
    new_text_data2 = chunk['second_phase'].astype(str)


    new_text_features = embed(new_text_data)
    new_text_features1 = embed(new_text_data1)
    new_text_features2 = embed(new_text_data2)

    new_text_features = np.round(new_text_features, 6)
    new_text_features1 = np.round(new_text_features1, 6)
    new_text_features2 = np.round(new_text_features2, 6)


    new_encoded_data = encoder.predict(new_text_features)
    new_encoded_data1 = encoder1.predict(new_text_features1)
    new_encoded_data2 = encoder2.predict(new_text_features2)

    new_encoded_data = np.round(new_encoded_data, 6)
    new_encoded_data1 = np.round(new_encoded_data1, 6)
    new_encoded_data2 = np.round(new_encoded_data2, 6)


    new_other_features = chunk.drop(columns=['processing', 'processing1', 'second_phase']).values

    new_other_features = np.round(new_other_features, 6)


    print("new_encoded_data shape:", new_encoded_data.shape)
    print("new_encoded_data1 shape:", new_encoded_data1.shape)
    print("new_encoded_data2 shape:", new_encoded_data2.shape)
    print("new_other_features shape:", new_other_features.shape)

    if new_encoded_data.shape[0] == new_encoded_data1.shape[0] == new_encoded_data2.shape[0] == new_other_features.shape[0]:

        new_combined_features = np.concatenate((new_encoded_data, new_encoded_data1, new_encoded_data2, new_other_features), axis=1)
        loaded_lgb_model = joblib.load('/content/drive/MyDrive/lgb_model_2.pkl')
        loaded_svr_model = joblib.load('/content/drive/MyDrive/svr_model_2.pkl')
        loaded_gpr_model = joblib.load('/content/drive/MyDrive/gpr_model_2.pkl')


        svr_predictions = loaded_svr_model.predict(new_combined_features)


        lgb_predictions = loaded_lgb_model.predict(new_combined_features)


        gpr_predictions = loaded_gpr_model.predict (new_combined_features)


        new_predictions = (svr_predictions + lgb_predictions + gpr_predictions)/3
        #new_predictions = (svr_predictions +  lgb_predictions)/2




        chunk['predicted_pw'] = new_predictions


        result1.append(chunk[['predicted_pw']])
    else:
        raise ValueError("所有输入数组的样本数量必须相同")


final_result1 = pd.concat(result1)

final_result1.to_csv('/content/drive/MyDrive/predicted_results_new_deformation_2024_1201.csv', index=False)

print("预测完成，结果已保存至: predicted_results_new.csv")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
new_encoded_data shape: (10000, 5)
new_encoded_data1 shape: (10000, 5)
new_encoded_data2 shape: (10000, 5)
new_other_features shape: (10000, 18)


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
new_encoded_data shape: (10000, 5)
new_encoded_data1 shape: (10000, 5)
new_encoded_data2 shape: (10000, 5)
new_other_features shape: (10000, 18)


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
new_encoded_data shape: (10000, 5)
new_encoded_data1 shape: (10000, 5)
new_encoded_data2 shape: (10000, 5)
new_other_features shape: (10000, 18)


...

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
new_encoded_data shape: (10000, 5)
new_encoded_data1 shape: (10000, 5)
new_encoded_data2 shape: (10000, 5)
new_other_features shape: (10000, 18)


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


255/255 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
255/255 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
255/255 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
new_encoded_data shape: (8150, 5)
new_encoded_data1 shape: (8150, 5)
new_encoded_data2 shape: (8150, 5)
new_other_features shape: (8150, 18)


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


预测完成，结果已保存至: predicted_results_new.csv
